In [ ]:
import numpy as np
import pandas as pd
import json
from sklearn.linear_model import LinearRegression
import os
from pprint import pprint
import requests
import urllib3
import ssl
from datetime import datetime

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
players_splits = None
players_info = None
opp_stats_G = None
opp_stats_A = None
opp_stats_C = None
with open('../data/full_players_splits.json') as f:
    players_splits = json.load(f)
with open('../data/players_filtered.json') as f:
    players_info = json.load(f)
with open('../data/opponent_stats_G.json') as f:
    opp_stats_G = json.load(f)
with open('../data/opponent_stats_F.json') as f:
    opp_stats_A = json.load(f)
with open('../data/opponent_stats_C.json') as f:
    opp_stats_C = json.load(f)

In [ ]:
def get_fg_percentage(str_miss):
    if type(str_miss) is float:
        return str_miss
    not_miss = float(str_miss.split('-')[0])
    total = float(str_miss.split('-')[1])
    if total == 0.0:
        return 0.0
    return (not_miss / total) * 100

In [ ]:
def clean_name(first_name_player, surname_player):
    return (first_name_player + " " + surname_player).strip().lower().replace(
        ".", "").replace('byombo',
                         'biyombo').replace('malcom', 'malcolm').replace(
                             'mohamed bamba', 'mo bamba').replace(
                                 'hilario nene', 'nene hilario').replace(
                                     'kelly oubre', 'kelly oubre jr').replace(
                                         'edrice adebayo', 'bam adebayo')

In [ ]:
def get_current_round():
    response = requests.get(
        'https://api.dunkest.com/api/rounds/current?league_id=1&fanta_league_id=1',
        verify=False)
    current_round = json.loads(response.text)
    return current_round['id']

def get_score_player(id_round, id_player):
    response = requests.get(
        'https://api.dunkest.com/api/players/' + str(id_player)
        + '/details?league_id=1&round_id=' + str(id_round),
        verify=False)
    current_round_stats = json.loads(response.text)
    return current_round_stats['score']

In [ ]:
players_resp = requests.get('https://api.dunkest.com/api/players?league_id=1',verify=False)
players_info_dunkest = json.loads(players_resp.text)
print(players_info_dunkest[0])

In [ ]:
i = 0
len_p = len(players_info_dunkest)
now = datetime.now()
players_info_dict = {}
id_round = get_current_round()
for player in players_info_dunkest:
    time_game = player['game_date']
    time_game = datetime.strptime(time_game, '%Y-%m-%d %H:%M:%S')
    if now > time_game and player['role_tag'] != "COACH" and player['lineup_tag'] != 'QUESTIONABLE'\
       and player['lineup_tag'] != 'INJURED' and player['lineup_tag'] != 'DISQUALIFIED'\
       and 'IN_DOUBT' not in player['lineup_tag']:
        first_name_player = player["name"]
        surname_player = player["surname"]
        name_player = clean_name(first_name_player, surname_player)
        print("{}   {} of {}".format(name_player, i, len_p))
        score = get_score_player(id_round, player['id'])
        players_info_dict[name_player] = [player['id'], player['opposing_team_name'].lower(),
                                          score, player['role_tag']]
        print("score: {}".format(score))
        i += 1
print("{} players computed, {} players skipped".format(i, len_p - i))

#for p in players_info_dunkest:
#    players_info_dict

In [ ]:
X_dataset = []
y_dataset = []
list_players = []
inverted_dict = {}

for player, splits in players_splits.items():
    if players_info_dict.get(player, None) is None:
        continue
    list_players.append(player)
    p_score = float(players_info_dict.get(player,None)[2])
    y_dataset.append(p_score)
    row_features = []
    if general_splits:
        del general_splits
    general_splits = splits['all splits']
    print(general_splits)
    general_splits[2] = get_fg_percentage(general_splits[2])
    general_splits[4] = get_fg_percentage(general_splits[4])
    general_splits[6] = get_fg_percentage(general_splits[6])
    vs_team = players_info_dict.get(player,None)[1]
    p_role = players_info[player][1]
    opp_stats = []
    row_features += general_splits
    #print(len(row_features))
    if p_role is 'G':
        opp_stats = opp_stats_G[vs_team]
    elif p_role is 'A':
        opp_stats = opp_stats_A[vs_team]
    elif p_role is 'C':
        opp_stats = opp_stats_C[vs_team]
    #print(len(opp_stats))
    row_features += opp_stats
    row_features = [float(elem) for elem in row_features]
    X_dataset.append(row_features)
    inverted_dict[sum(row_features)] = player
    print(row_features)
    print("=" * 60)
    #print(len(row_features))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, test_size=0.2, train_size=0.8, random_state=0)

In [ ]:
model = LinearRegression().fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error

for elem in range(len(y_pred)):
    print(inverted_dict[sum(X_test[elem])])
    print(y_test[elem], y_pred[elem])

print(mean_squared_error(y_test, y_pred))